<a href="https://colab.research.google.com/github/Matthieu6/IndividualProject/blob/main/ClipScore_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialise


This section installs the required extensions to run the CLIP Score metrics

In [ ]:
!pip install -q datasets diffusers transformers accelerate torchmetrics[image]
!pip install Pillow

## Find CLIP Score for list of Images


This section determines the CLIP score of images kept in multiple folders under the same directory. Each image is assessed with the 5 reference captions found on the Flickr8k database. The reference caption - image pairs are tested separately in each folder, which is then averaged to find the average CLIP score for each folder.

In [ ]:
import torch
from torchmetrics.functional.multimodal import clip_score
from functools import partial
import numpy as np
from PIL import Image
import io
import os

# Function to calculate CLIP score
clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(image, prompts):
    image_int = (image * 255).astype("uint8")
    clip_score_value = clip_score_fn(torch.from_numpy(image_int).permute(2, 0, 1).unsqueeze(0), prompts).detach()
    return round(float(clip_score_value), 4)

# Define reference image captions obtained from the Flickr8k Database

prompts_list = [
    ["A girl paddling down a large river , as seen from behind her", "A woman kayaking down a river" ,"A woman paddles a boat down a river","A woman with a paddle","A woman with a paddle sits in a boat on the water"],
    ["A lone skier is making their way up a mountain", "Skiiers at the top of a snow covered hill ", "Two hikers climbing a snowy hill","Two people are on a snow-covered mountain","Two people climbing up a snowy mountain"],
    ["A large man sits in the back of a canoe with another man in front on a river","two man are paddling a kayak along a river with green trees on either side","Two men in a small boat rowing down a river","Two men paddling a canoe down a narrow stream","Two paddlers propel a canoe down a tree-lined stream"],
    ["A couple enjoys a quiet moment near the river","A few people sit out on rocks overlooking the water below","Two people are sitting on a rock overlooking the river","Two people sit on top of a cliff overlooking a river","Two people sit upon large rocks next to a river"],
    ["A couple sit on the hood of a car with their backs to the camera","A couple sits on a car","A man and a woman sit on a car and face the opposite way of the camera","A man and woman sit on a car and watch the road","A man and woman sitting on a parked car"],
    ["A motorcycle racer leans his bike","A motorcyclist is driving down a road on their motorbike","A motorcyclist is riding their sponsored car along a roadway that has recently turned","A motorcyclist on the street","A motorcyclist with a red helmet rides his blue motorcycle down the road"],
    ["A mountain climber looks down from the summit","A person is rock climbing between two large rock faces","A rock climber climbs in between two very large rocks","A rock climber leans out from a crag against a deep blue sky","Two large mountains stand high in the air as a person tries to climb one"],
    ["A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl", "A little girl is sitting in front of a large painted rainbow","A small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on it","There is a girl with pigtails sitting in front of a rainbow painting","Young girl with pigtails painting outside in the grass"],
    ["A brown dog chases the water from a sprinkler on a lawn", "a brown dog plays with the hose","A brown dog running on a lawn near a garden hose","A dog is playing with a hose", "Large brown dog running away from the sprinkler in the grass" ],
    ["A family playing on a tractor on a beautiful day","Children ride a tractor in a field","Several children playing on a Polaris vehicle outdoors","The children played on the four wheeler in the field","The four kids are riding on an ATV in a field"],
    ["Two little puppies chase each other on a stone wall", "Two puppies run across flat stones in garden area","Two puppies walking on stones","Two small puppies run along the edge of a stone wall nearby a heavily foliaged area","two small tan puppies walking on a stone path"],
    ["a lone woman sitting on an usual green bench with a cup in her hand","A woman sits on a flower pot at a crowded coffee shop","The woman sat on a green bench and had a drink","Woman holding a drink with a white bag next to her is sitting on a green sculpture","Woman sitting on a turquoise bench with a pillow and a bag near a restaurant"],
    ["A lone man stands on a rocky outcropping and looks out over the wilderness","A man in shorts and a black jacket stands on top of a big rock looking down on hills and valleys","A man is standing on a big rock overlooking a valley","A man stands atop a large mountain of rocks","Man standing on edge of large rock overlooking scenic view"],
    ["A football player pauses during a game","A football player wears a green jersey with the number "" 4 "" on it","Greenbay football player is being handed a towel on the field","Green Bay Packer player cooling off","Someone takes a cloth off of a Green Bay Packers football player"],

    # Add more sets of prompts as needed
]

# Directory containing folders of images to be compared with reference captions
parent_dir = "/content/drive/MyDrive/ImageReferences"

# List subdirectories (folders) in the parent directory
image_folders = [folder_name for folder_name in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, folder_name))]

# Sort folders
image_folders.sort()

image_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

# Iterate through each folder
for folder_name in image_folders:
    folder_path = os.path.join(parent_dir, folder_name)
    print(f"\nProcessing images in folder: {folder_path}")

    # List image files in the current folder
    image_files = os.listdir(folder_path)
    image_files.sort()  # Sort the image files

    # Store average scores for all images in the current folder
    folder_average_scores = []

    # Iterate through each image file in the folder
    for i, image_index in enumerate(image_indices, start=1):
        image_file = f"output_image_{image_index}.png"
        image_path = os.path.join(folder_path, image_file)
        print(f"\nProcessing image: {image_path}")

        # Load the input image
        with open(image_path, "rb") as f:
            image_data = f.read()
        image = Image.open(io.BytesIO(image_data))
        image_np = np.array(image)

        # Calculate CLIP score for the input image against the corresponding set of prompts
        prompts = prompts_list[i - 1]  # Get prompts for current image
        clip_scores = []
        for prompt in prompts:
            clip_score_value = calculate_clip_score(image_np, [prompt])
            clip_scores.append(clip_score_value)

        # Calculate and print average score for the current image
        average_score = sum(clip_scores) / len(clip_scores)
        print(f"Average CLIP score for {image_file}: {average_score}")

        # Append average score to folder_average_scores list
        folder_average_scores.append(average_score)

    # Calculate and print average score for all images in the folder
    folder_overall_average_score = sum(folder_average_scores) / len(folder_average_scores)
    print(f"\nOverall average CLIP score for all images in {folder_name}: {folder_overall_average_score}")
